In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Taken from the EDA notebook
from os import listdir
from os.path import isfile, join
def get_filenames(mypath):
    return [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
filenames = ["../data/audioset_v1_embeddings/bal_train/" + i for i in get_filenames("../data/audioset_v1_embeddings/bal_train/")]
train_dataset = tf.data.TFRecordDataset(filenames)

# Make Pandas DataFrame out of the training set records.
col = ['video_id', 'time_stamp'] + [str(k) for k in range(0,128)]

placeholder_array = np.array(col)

for raw_record in train_dataset:
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())
    for i in range(0,len(example.feature_lists.feature_list['audio_embedding'].feature)):
        vID = example.context.feature['video_id'].bytes_list.value[0]
        time = example.context.feature['start_time_seconds'].float_list.value[0] + 0.96*i
        placeholder_array = np.vstack((placeholder_array, np.array([vID, time] + list(example.feature_lists.feature_list['audio_embedding'].feature[0].bytes_list.value[0]))))

print('Done making the array. Converting to Pandas DF and saving.')

trainFeatures = pd.DataFrame(np.delete(placeholder_array, 0, 0), columns = col)

trainFeatures.to_csv('trainFeatures.csv')
trainFeatures.head()

In [39]:
# Finally, make a Pandas DF for the target: whether or not speech is present in each 0.96 second chunk.
trainTargets = pd.read_csv('trainFeatures.csv', header = 0, index_col = 0).iloc[:,0:2]
trainTargets['speech_present'] = False # By default.
print(trainTargets.head())

#trainTargets = trainFeatures[:,:'time_stamp']
#del TrainFeatures # For RAM's sake. We've already saved this to a CSV.

# Make a Pandas DataFrame of all instances of speech present.
trainEvents = pd.read_csv("../data/audioset_train_strong.tsv", sep="\t")
print(trainEvents.head())

''' Have to make the labels match the feature set,
    and these labels have a "_" followed by trailing digits.'''
trainEvents.iloc[:,0] = trainEvents.iloc[:,0].str.rstrip("0123456789")
trainEvents.iloc[:,0] = trainEvents.iloc[:,0].str.rstrip("_")
print(trainEvents.head())

filter = set(trainTargets.iloc[:,0]).intersection(set(trainEvents.iloc[:,0]))
print(filter)

for i in range(0,len(trainTargets)):
    if (trainTargets.iloc[i,0] in filter) == False:
        trainTargets.iloc[i,0] = None
trainTargets = trainTargets.dropna() # Deletes all rows with a None in it, i.e. entries not in the intersection.

trainTargets.to_csv('trainTargetsFiltered.csv')



      video_id  time_stamp  speech_present
0  --cB2ZVjpnA       30.00           False
1  --cB2ZVjpnA       30.96           False
2  --cB2ZVjpnA       31.92           False
3  --cB2ZVjpnA       32.88           False
4  --cB2ZVjpnA       33.84           False
          segment_id  start_time_seconds  end_time_seconds       label
0  b0RFKhbpFJA_30000               0.000            10.000  /m/03m9d0z
1  b0RFKhbpFJA_30000               4.753             5.720   /m/05zppz
2  b0RFKhbpFJA_30000               0.000            10.000  /m/07pjwq1
3  b0RFKhbpFJA_30000               6.899             7.010  /m/07qjznt
4  b0RFKhbpFJA_30000               8.534             9.156  /t/dd00092
    segment_id  start_time_seconds  end_time_seconds       label
0  b0RFKhbpFJA               0.000            10.000  /m/03m9d0z
1  b0RFKhbpFJA               4.753             5.720   /m/05zppz
2  b0RFKhbpFJA               0.000            10.000  /m/07pjwq1
3  b0RFKhbpFJA               6.899             7.010  /m

KeyboardInterrupt: 

In [43]:
print('Current size of trainEvents is:' + str(len(trainEvents)))
for i in range(0,len(trainEvents)):
    if (trainEvents.iloc[i,0] in filter) == False:
        trainEvents.iloc[i,0] = None
trainEvents = trainEvents.dropna() # Deletes all rows with a None in it, i.e. entries not in the intersection.

trainEvents.to_csv('trainEventsFiltered.csv')

trainFeatures0 = pd.read_csv('trainFeatures.csv', header = 0, index_col = 0)
for i in range(0,len(trainFeatures0)):
    if (trainFeatures0.iloc[i,0] in filter) == False:
        trainFeatures0.iloc[i,0] = None
trainFeatures0 = trainFeatures0.dropna() # Deletes all rows with a None in it, i.e. entries not in the intersection.

trainFeatures0.to_csv('trainFeaturesFiltered.csv')

Current size of trainEvents is:474570


C:\Users\Jacob\anaconda3\envs\hello-tf\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
trainTargets = pd.read_csv('trainTargetsFiltered.csv', header = 0, index_col = 0)
trainEvents = pd.read_csv('trainEventsFiltered.csv', header = 0, index_col = 0)

# Respectively, speech, male speech, female speech, child speech, conversation, narration, babbling, synthetic speech.
speech_events = set(['/m/09x0r', '/m/05zppz','/m/02zsn','/m/0ytgt','/m/01h8n0','/m/02qldy','/m/0261r1','/m/0brhx'])

# Now check to see if each 0.96 second segment contains speech according to the trainEvents DF.
'''This seems complicated at first glance, but the idea behind it is simple.
Since each clip's events are grouped together in the trainEvents TSV, we just
need to run a search ONCE for each clip label. Once we have it, we don't need
to search again for the next entry's events unless its label is different.
Sadly, the labels are NOT in alphabetical order, making an approach like this
necessary.'''

# Apparently the .96 second annotations were only applied to 5% of the overall dataset.
# There are elements of the balanced dataset that are missing...
no_match = np.array(None) 

first_label_match = 0
for i in range(0,10):
    if first_label_match == 0:
        while (trainTargets.iloc[i,0] != trainEvents.iloc[first_label_match,0]) and (first_label_match <= len(trainTargets)):
            first_label_match += 1
            if first_label_match == len(trainTargets):
                print(trainTargets.iloc[i,0] + ' has no match.')
                no_match = np.concatenate((no_match, [trainTargets.iloc[i,0]]))
    offset = 0
    while (trainTargets.iloc[i,0] == trainEvents.iloc[first_label_match + offset,0]):
        if trainTargets.iloc[i,1] <= trainEvents.iloc[first_label_match + offset,1]:
            if trainTargets.iloc[i,1] + 0.96 >= trainEvents.iloc[first_label_match + offset,1]:
                trainTargets.iloc[i,2] = True
        if trainTargets.iloc[i,1] >= trainEvents.iloc[first_label_match + offset,1]:
            if trainTargets.iloc[i,1] <= trainEvents.iloc[first_label_match + offset,2]:
                trainTargets.iloc[i,2] = True
        offset += 1
    if i != len(trainTargets) - 1:
        if trainTargets.iloc[i,0] != trainTargets.iloc[i+1,0]:
            first_label_match = 0

trainTargets.to_csv('trainTargets.csv')

np.savetxt('missing_segments.csv', no_match, delimiter=',')
# When done, we can drop the labels and time indices, since the orders are the same.

      video_id  time_stamp  speech_present
0  --cB2ZVjpnA       30.00           False
1  --cB2ZVjpnA       30.96           False
2  --cB2ZVjpnA       31.92           False
3  --cB2ZVjpnA       32.88           False
4  --cB2ZVjpnA       33.84           False
          segment_id  start_time_seconds  end_time_seconds       label
0  b0RFKhbpFJA_30000               0.000            10.000  /m/03m9d0z
1  b0RFKhbpFJA_30000               4.753             5.720   /m/05zppz
2  b0RFKhbpFJA_30000               0.000            10.000  /m/07pjwq1
3  b0RFKhbpFJA_30000               6.899             7.010  /m/07qjznt
4  b0RFKhbpFJA_30000               8.534             9.156  /t/dd00092
    segment_id  start_time_seconds  end_time_seconds       label
0  b0RFKhbpFJA               0.000            10.000  /m/03m9d0z
1  b0RFKhbpFJA               4.753             5.720   /m/05zppz
2  b0RFKhbpFJA               0.000            10.000  /m/07pjwq1
3  b0RFKhbpFJA               6.899             7.010  /m

00mE-lhe_R8 has no match.
01hjVJN9xCg has no match.
01k1v-NgjWs has no match.
01xpKyI0rXA has no match.
01bTS8O2Xnc has no match.
01q8wKX4XEQ has no match.
01B907_Gyys has no match.
0150dZu3Na8 has no match.
01PzcPKT3_E has no match.
0298WjE3_tk has no match.
02Qntw26enM has no match.
021B2ozAjNg has no match.
02z_7DUC_74 has no match.
02tl_ek8f9I has no match.
02rLonMSO9k has no match.
02aq42RDm2Q has no match.
02pl9PaQwmI has no match.
02QXbqf9bbw has no match.
02JEvDAcq7U has no match.
02R_w3cr1i4 has no match.
02n2I1JHq4U has no match.
02q5C77_xeQ has no match.
032FsMMKWqM has no match.
03kFLTZguBs has no match.
03NLMEMi8-I has no match.
03EmZCCkf-E has no match.
03lAXcmqsMU has no match.
03frQGyrgQ4 has no match.
03z0rpIkm5g has no match.
04SFLoexHDo has no match.
046ynbCwM90 has no match.
04RDcHpgzTM has no match.
04JKX_vlePE has no match.
05H3CBB2oiI has no match.
058Nv060Fz4 has no match.
05OJDYeHLMc has no match.
05wEdbchyjM has no match.
06Brdf83RZE has no match.
06J19koRu_4 

194N_h5CGzA has no match.
19Pp9QEw17U has no match.
19DRe0cdeXQ has no match.
19tPF3TY3g0 has no match.
19-GI2LzOtc has no match.
192AgE8E-cw has no match.
1aiMcQqOIO4 has no match.
1aLTEOXlRzU has no match.
1a6-xAO5Dik has no match.
1a6ziKisD4Q has no match.
1aVhpF7Sm8o has no match.
1aNc3AkgJNU has no match.
1aulP-srFAI has no match.
1alQZjvWr38 has no match.
1aavDifdIAk has no match.
1aEh44p0EqY has no match.
1bsXgn_25VQ has no match.
1bOukvpECmU has no match.
1bet-VrOUlM has no match.
1blEjTegTzY has no match.
1caCiNiUErA has no match.
1c5rSQsqmBE has no match.
1c7NBk1x3y0 has no match.
1cFHCB6bR8I has no match.
1cS0oGvV5PY has no match.
1c8yp4_Fji8 has no match.
1cjduhFyTkA has no match.
1cew7dAbDh0 has no match.
1DvlbESnFjo has no match.
1DiSaLjlU80 has no match.
1DH7AhyL0KU has no match.
1DxLwZhTj0A has no match.
1eQSkR2w0R8 has no match.
1eW-T6vtO7k has no match.
1e3_ucOz0Ik has no match.
1ei2Ha64te4 has no match.
1ecvyHKH9TI has no match.
1et9yT2BYD0 has no match.
1ejb5TtkhY8 

2IWV8hlUydk has no match.
2I9ToRyfL5k has no match.
2juN7xQerfw has no match.
2jraQyyG0qY has no match.
2jW4P4mGxo8 has no match.
2j8hxco6pBk has no match.
2jFE-MKRmPw has no match.
2je_d-3I0OY has no match.
2jIxHKpgBys has no match.
2K61zdPjAi8 has no match.
2KgX6IvEVrc has no match.
2K5j6V0vsvM has no match.
2KkNk9Ao7G4 has no match.
2KwwQHit0-Q has no match.
2KSST-I4Fj8 has no match.
2KK9MldzvzE has no match.
2K8O1KQQw54 has no match.
2lBZ6yPW9WU has no match.
2lXyUsQZ5Bo has no match.
2mOwAL8_vog has no match.
2mXZfaHadmU has no match.
2mEH2FUWUTg has no match.
2mbrbHlsBgk has no match.
2mF3jZwPXSc has no match.
2mJbGx5D-zA has no match.
2mBef9KrCpk has no match.
2mAbc1JnXXY has no match.
2nd27vVGg8k has no match.
2nADZL8olrg has no match.
2n13-18I_nM has no match.
2neeSM0lvbs has no match.
2nsUnuIkzDE has no match.
2nS_rk1118M has no match.
2nbyIEmQAaU has no match.
2Og-RS5LXno has no match.
2O22edWULCE has no match.
2O4TajdF_Cc has no match.
2OROEX0O-kw has no match.
2OEm9kmVgIc 

3VEMHWnewuc has no match.
3V4-O2oVF-M has no match.
3Wknf4Exf0w has no match.
3WQYwScCntc has no match.
3WBvhwIEAEI has no match.
3WUwmsGEozE has no match.
3W6hY1G86c8 has no match.
3WkbOuahBnk has no match.
3W9GRWaRJPo has no match.
3W6ajTt2UzU has no match.
3xCWI_22Z9A has no match.
3xHttYIwocY has no match.
3xFqT_fegTo has no match.
3YGB2o-lw6w has no match.
3YpDove6j8Q has no match.
3Y-VT4IZtsc has no match.
3Yc7_n6mDsI has no match.
3YzsvHJ10VY has no match.
3YJDtqVya3g has no match.
3YqNm3QaCjk has no match.
3YIGFVl4Euk has no match.
3Zt4Y3Jajrk has no match.
3ZUHhSbQkGw has no match.
3Z67jXHxHow has no match.
3ZEzvNaHWPE has no match.
3ZzEYwxiWZY has no match.
3ZhyXbwFQAM has no match.
3ZDpqDpIsfs has no match.
3ZyuBJEbmJM has no match.
3Z0Gl7RHThc has no match.
3_ePrXeNxvQ has no match.
3_58wEMlFyk has no match.
3_14lh_jd4s has no match.
4-euhrtcpXQ has no match.
4-7xip9t_5M has no match.
4-MVa_JAapU has no match.
40i3_JH6FYw has no match.
40kbMyL2Wgo has no match.
40mnKuqicts 

...

uKSH2Nc04uc has no match.
uKebpQMHWVI has no match.
ULS3ffQkCW4 has no match.
ULHPhjaJ6p0 has no match.
ULW0iSZ7kpc has no match.
ULFhHR0OLSE has no match.
ULXbXpLcoVA has no match.
UL7tkfWe9Bc has no match.
UmEPR7gTIb0 has no match.
Um91Y1Vy5AU has no match.
UmminfCXhNw has no match.
UmlLEH2fpsA has no match.
UmOOGxXbEuc has no match.
UmAZXAdkstk has no match.
UmbVKqmvaRY has no match.
uNPUAFx8tyg has no match.
uNV9hUiAdeo has no match.
uN0Wt4qOxps has no match.
uNIuDkhwodA has no match.
uNfvtOokcLM has no match.
uovtUHXztoI has no match.
uoQaqXTGFwc has no match.
uovp63FkFaY has no match.
uoZ9Rs4E-6Y has no match.
uoirx7CU79E has no match.
UpSoM6ZJRyg has no match.
UpWBYk4xLTc has no match.
UpYkuf0greU has no match.
uq9kej0JLb4 has no match.
uqAY4lCUcRY has no match.
uqtwOTsOF7I has no match.
uqr2NRv0LOY has no match.
uqUNGNZoxrg has no match.
uRIzEtwt8MY has no match.
uRwlP0A57Oo has no match.
uRA5Ue30fsY has no match.
uRZKIy-a39Q has no match.
uS2kPLvEo6A has no match.
uSItpibTSVk 

WdhAku9Codg has no match.
WditOomsdRU has no match.
WdxNikVdK-U has no match.
WdTzXm3ntZY has no match.
Wd4T3iTsgrI has no match.
WdD4TtenRPA has no match.
wE9Wmfcscic has no match.
wEZ5_0Mm6yI has no match.
wEHRmd84Dwc has no match.
wECGYE_1TGU has no match.
wELDS_DgC84 has no match.
wEqqUlchZ7U has no match.
wEwGIryoU0U has no match.
wEmEkLQDl0U has no match.
wE5_OTgj51E has no match.
WfehVmSEyLQ has no match.
Wf_wfMaRAkI has no match.
wgkITtt6jSg has no match.
wg5q4ub74yc has no match.
wgF9Z-Ui6x8 has no match.
wgcsA-YWCaQ has no match.
wgQ96FJceRQ has no match.
wgKWQMR2yUc has no match.
wgaovDRpLQM has no match.
WHW_r9SFmIs has no match.
WHvURI0YXqc has no match.
WHFFyxCgN9Y has no match.
WHLbGh7WwNY has no match.
WH9D6yBJwQk has no match.
WHSe2LcuCdg has no match.
wi8EhPUIVTg has no match.
wiDw2V0pOS0 has no match.
wiWxg1qAqJ4 has no match.
WJaWSIC2m8k has no match.
WJ0ZY5KzjJo has no match.
WJcaxEPB3lI has no match.
wkkzzAjSxDs has no match.
wkQgTPAldgE has no match.
wlkSmyEWNX0 

y5CzboExtAk has no match.
y5a05m8MD3k has no match.
y5mdozqiREI has no match.
y5q8M5iRS4A has no match.
y63vbjQjajs has no match.
y6iMm7Pltq0 has no match.
y62kbFHfdX0 has no match.
y6jEec1e6Gw has no match.
y6EpGVIotA0 has no match.
y6tnyCHq6xE has no match.
y6R58_iLdJA has no match.
y6aPTI3QuMY has no match.
y75AAF1JtZw has no match.
y72M4-s0HEw has no match.
y7nvhaY-A58 has no match.
y7PIXFAWeLk has no match.
y7BLTK9l45g has no match.
Y8sHHzZ6bS8 has no match.
Y8plCTF8xsY has no match.
Y8i-j0cX9nY has no match.
Y8gs9hvqRUY has no match.
Y8f_2TVBSU8 has no match.
Y8svkK7d7sY has no match.
Y8AB0YA4Jtw has no match.
Y93yBzxqtJ0 has no match.
Y9ys-Yx5_j0 has no match.
Y9iRveRVZpQ has no match.
Y9YhG2CFTz0 has no match.
Y9mLvKjYZfc has no match.
Y9VbO6tYiBQ has no match.
YAVVMxqtxGE has no match.
YAYp2E5vMNw has no match.
YABeQ0ipHf0 has no match.
YAhRe80ue28 has no match.
YAubvoQv7uI has no match.
YbVaX67JHnQ has no match.
yc_hl6Y7a-Y has no match.
yc_bzicMBzg has no match.
ycXARBCdxBg 

ZUPJpA69H_8 has no match.
ZUg7rRpFGvA has no match.
zVA66aZOBH4 has no match.
zVuH6RSpvyY has no match.
zVLqS6y0yxE has no match.
zWPwtay0dxM has no match.
zWQJrAaHsFg has no match.
zWqvlnU-Kq8 has no match.
zWw2rnj7t5w has no match.
zWOD_bnFjF8 has no match.
zWEznZ40k2Q has no match.
zWmF_2u-9nk has no match.
zxBo7nnAMVE has no match.
zx3zeFZR7CA has no match.
zx19pQlKn5I has no match.
zxnbSCXxwb4 has no match.
zx0XSGhf7m0 has no match.
zxZLB2AJj3A has no match.
zxeeysNrEvM has no match.
zY5DN4MOv7Q has no match.
zY2E_NvRow0 has no match.
zY2JAbEVwGY has no match.
zYa3ksSrCqw has no match.
zYL0yt3bFcc has no match.
zY1LJu1j4zw has no match.
zYRI-R2gxUI has no match.
zY9Bw7oW7Uo has no match.
zYsFbF9emtI has no match.
zYmAbrAZDy0 has no match.
zYlOfvuzbBo has no match.
zzya4dDVRLk has no match.
zzlK8KDqlr0 has no match.
zz8TGV83nkE has no match.
zz0ddNfz0h0 has no match.
z_mf1ceM8jc has no match.
z_0jJiM7nyU has no match.
z_iUAQCVLwo has no match.
z_fEIZOv0JY has no match.
_-lXVZ9QpO8 

AttributeError: module 'numpy' has no attribute 'savetext'

In [24]:
print(trainTargets.iloc[0,0])
print(type(trainTargets.iloc[0,0]))
print(len(trainTargets.iloc[0,0]))
print(trainEvents.iloc[0,0])
print(type(trainEvents.iloc[0,0]))
print(len(trainEvents.iloc[0,0]))

--cB2ZVjpnA
<class 'str'>
11
b0RFKhbpFJA
<class 'str'>
11


In [ ]:
# Do the same for the eval DS...
filenames = ["../data/audioset_v1_embeddings/eval/" + i for i in get_filenames("../data/audioset_v1_embeddings/bal_train/")]
eval_dataset = tf.data.TFRecordDataset(filenames)


In [ ]:
# This was copy-pasted from my old personal project.
# Some of the layers are not applicable (normalization),
# and some layers (input, reshape, resizing, and dense) need to be resized.
# shape = (,), (leftover from the input layer)

model = Sequential([
    layers.Input(ragged=True),
    layers.Conv2D(4, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(4, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(200, activation='relu'),
    layers.Dense(160, activation='relu'),
    layers.Dense(120, activation='relu'),
    layers.Dense(80, activation='relu'),
    layers.Dense(37, activation='relu')
])

model.summary()

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mean_squared_error'])
model.fit(trainFeatures,trainTargets,epochs = 100, batch_size = 4)
if save_to_disk:
    # Saving model to JSON and weights to H5.
    os.chdir('..')
    os.chdir('./models/')
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")
loss  = model.evaluate(ValFeatures, ValTargets)
print('Loss on Validation Set: ', loss)